In [ ]:
import boto3
import os
from mlflow.deployments import get_deploy_client
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
boto3.setup_default_session(
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name=os.getenv('SAGEMAKER_REGION')
)

In [ ]:
model_uri = "/home/erik/DataspellProjects/Stock-Prediction/mlruns/467592723741106657/c413fe91b94f4a8db7492ffa6657a0f6/artifacts/model"

aws_account_id = "401605265667"
execution_role_arn = "arn:aws:iam::401605265667:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole"

tag_id = '2.8.1'
image_url = "401605265667.dkr.ecr.eu-central-1.amazonaws.com/for-sagemaker-deployment:" + tag_id

config = {
    "execution_role_arn": execution_role_arn,
    "bucket_name": "xgboostbucket",
    "image_url": image_url,
    "region_name": os.getenv('SAGEMAKER_REGION'),
    "archive": False,
    "instance_type": "ml.m5.large", # ml.t3.medium
    "instance_count": 1,
    "synchronous": True,
    "timeout_seconds": 3600,
    "variant_name": "prod-variant-1",
    "tags": {"training_timestamp": "2023-11-22"},
}

client = get_deploy_client("sagemaker")

client.create_deployment(
    name="stockmodel",
    model_uri=model_uri,
    flavor="python_function",
    config=config,
)

In [ ]:
## AWS Container
# mlflow sagemaker build-and-push-container --build --push -c for-sagemaker-deployment

## Deploy Model
# mlflow sagemaker deploy --app-name xformer -m runs:/7-16420961991120000000062/model 
# --execution-role-arn arn:aws:iam::612652722220:role/role-for-mlflow-sagemaker-deploy 
# --bucket jaganes-sagemaker 
# --image-url 612652722220.dkr.ecr.us-east-1.amazonaws.com/for-sagemaker-deployment:1.22.0 
# --region-name us-east-1 
# --mode create 
# --instance-type ml.m5.large 
# --instance-count 1 
# --flavor python_function